In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

df = pd.read_csv('/Users/akounoroushi/Desktop/claims_final.csv')
df.columns = ['Family ID', 'Member ID', 'Provider ID', 'Provider Type', 'State', 'Date', 'Procedure', 'Amount']
df.head()

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount
0,327546,0,2299,122398719,NY,20060526,280,381.37
1,342944,5,5367,122398782,NY,20060817,1688,889.29
2,342944,5,5367,122398782,NY,20060717,1688,889.29
3,72526,0,9029,122398719,NY,20060221,562,1681.68
4,234943,1,10215,122398719,FL,20060615,288,336.63


In [3]:
# Get an unique ID for each patient
df['Patient ID'] = df['Family ID'] * 10 + df['Member ID']
df.head()

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount,Patient ID
0,327546,0,2299,122398719,NY,20060526,280,381.37,3275460
1,342944,5,5367,122398782,NY,20060817,1688,889.29,3429445
2,342944,5,5367,122398782,NY,20060717,1688,889.29,3429445
3,72526,0,9029,122398719,NY,20060221,562,1681.68,725260
4,234943,1,10215,122398719,FL,20060615,288,336.63,2349431


In [4]:
# Normalize the amount of claimed money
df['amountLog'] = df['Amount'].apply(np.log)
df['Amount Normed'] = df.groupby(['Provider Type', 'Procedure'], group_keys=False)\
    .apply(lambda g: (g['amountLog'] - g['amountLog'].mean()) / g['amountLog'].std())
df.head()

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount,Patient ID,amountLog,Amount Normed
0,327546,0,2299,122398719,NY,20060526,280,381.37,3275460,5.943770,-0.095999
1,342944,5,5367,122398782,NY,20060817,1688,889.29,3429445,6.790423,-0.734812
2,342944,5,5367,122398782,NY,20060717,1688,889.29,3429445,6.790423,-0.734812
3,72526,0,9029,122398719,NY,20060221,562,1681.68,725260,7.427549,0.696837
4,234943,1,10215,122398719,FL,20060615,288,336.63,2349431,5.818984,-1.315975


In [7]:
# Normatize the frequency of each provider gives the same procedure to the same patient
df['patientCount'] = df.groupby(['Provider Type', 'Procedure', 'Patient ID'])['Patient ID'].transform('count')
df['patientCountLog'] = df['patientCount'].apply(np.log)
df['Patient Count Normed'] = df.groupby(['Provider Type', 'Procedure'], group_keys=False)\
    .apply(lambda g: (g['patientCountLog'] - g['patientCountLog'].mean()) / g['patientCountLog'].std())
df.head()

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount,Patient ID,amountLog,Amount Normed,patientCount,patientCountLog,Patient Count Normed,providerCount,providerCountLog,Provider Count Normed
0,327546,0,2299,122398719,NY,20060526,280,381.37,3275460,5.943770,-0.095999,1,0.000000,-0.519923,146719,11.896274,0.999997
1,342944,5,5367,122398782,NY,20060817,1688,889.29,3429445,6.790423,-0.734812,2,0.693147,-1.563360,32498,10.388934,-0.999985
2,342944,5,5367,122398782,NY,20060717,1688,889.29,3429445,6.790423,-0.734812,2,0.693147,-1.563360,32498,10.388934,-0.999985
3,72526,0,9029,122398719,NY,20060221,562,1681.68,725260,7.427549,0.696837,1,0.000000,-0.399465,1008,6.915723,0.999504
4,234943,1,10215,122398719,FL,20060615,288,336.63,2349431,5.818984,-1.315975,1,0.000000,-0.710111,152920,11.937670,0.999997


In [8]:
# Normatize the frequency of each patient receives the same procedure from the same type of provider
df['providerCount'] = df.groupby(['Provider Type', 'Procedure', 'Provider ID'])['Provider ID'].transform('count')
df['providerCountLog'] = df['providerCount'].apply(np.log)
df['Provider Count Normed'] = df.groupby(['Provider Type', 'Procedure'], group_keys=False)\
    .apply(lambda g: (g['providerCountLog'] - g['providerCountLog'].mean()) / g['providerCountLog'].std())
df.head()

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount,Patient ID,amountLog,Amount Normed,patientCount,patientCountLog,Patient Count Normed,providerCount,providerCountLog,Provider Count Normed
0,327546,0,2299,122398719,NY,20060526,280,381.37,3275460,5.943770,-0.095999,1,0.000000,-0.519923,55,4.007333,-0.536115
1,342944,5,5367,122398782,NY,20060817,1688,889.29,3429445,6.790423,-0.734812,2,0.693147,-1.563360,392,5.971262,0.157313
2,342944,5,5367,122398782,NY,20060717,1688,889.29,3429445,6.790423,-0.734812,2,0.693147,-1.563360,392,5.971262,0.157313
3,72526,0,9029,122398719,NY,20060221,562,1681.68,725260,7.427549,0.696837,1,0.000000,-0.399465,3,1.098612,-0.205038
4,234943,1,10215,122398719,FL,20060615,288,336.63,2349431,5.818984,-1.315975,1,0.000000,-0.710111,44,3.784190,-0.262850


In [16]:
# Features used hare are the frequency of each patient receives the same procedure from the same type of provider, 
# the frequency of each provider gives the same procedure to the same patient, and the amount of money
lof = LocalOutlierFactor()
df = df.dropna()
grouped = df.groupby(['Provider Type', 'Procedure'])
newDF = pd.DataFrame()
for name, group in grouped:
    lof.fit(group[['Amount Normed', 'Patient Count Normed', 'Provider Count Normed']])
    # Get LOF scored based on features. The smaller the score is, the more abnormal it is. We may sort by this score.
    score = getattr(lof, 'negative_outlier_factor_')
    group['Score'] = score
    newDF = newDF.append(group)
newDF.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/lof.py:180: UserWarning: n_neighbors (20) is greater than the total number of samples (13). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/lof.py:180: UserWarning: n_neighbors (20) is greater than the total number of samples (15). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/lof.py:180: UserWarning: n_neighbors (20) is greater than the t

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount,Patient ID,amountLog,Amount Normed,patientCount,patientCountLog,Patient Count Normed,providerCount,providerCountLog,Provider Count Normed,Score
1627,196288,3,8886,122398719,NY,20060707,83,822.62,1962883,6.712494,0.386699,1,0.0,-0.054554,3,1.098612,-0.040139,-1.000000e+00
2043,286170,2,3775,122398719,CA,20060720,83,795.15,2861702,6.678531,0.266175,1,0.0,-0.054554,3,1.098612,-0.040139,-1.578265e+08
2481,490435,2,6848,122398719,NY,20060803,83,822.62,4904352,6.712494,0.386699,1,0.0,-0.054554,10,2.302585,1.464829,-1.000000e+00
2763,486863,2,14337,122398719,NY,20060212,83,735.78,4868632,6.600931,-0.009197,1,0.0,-0.054554,3,1.098612,-0.040139,-1.000000e+00
2904,150295,3,15861,122398719,NY,20060725,83,816.46,1502953,6.704978,0.360026,1,0.0,-0.054554,7,1.945910,1.018985,-1.000000e+00


In [23]:
grouped = newDF.groupby('Provider Type')
ranked = pd.DataFrame()
for name, group in grouped:
    group = group.sort_values(by=['Score'])
    group['rank'] = range(1, len(group.index)+1)
    ranked = ranked.append(group)
ranked.head()

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount,Patient ID,amountLog,Amount Normed,patientCount,patientCountLog,Patient Count Normed,providerCount,providerCountLog,Provider Count Normed,Score,rank
1173274,265506,1,18293,122398719,NY,20060605,153,976.19,2655061,6.883657,0.188191,2,0.693147,10.062427,90,4.499810,1.526027,-8.637489e+10,1
2814419,265506,1,18293,122398719,NY,20060605,153,976.19,2655061,6.883657,0.188191,2,0.693147,10.062427,90,4.499810,1.526027,-8.637489e+10,2
3215503,259922,0,4421,122398719,NY,20060807,168,651.31,2599220,6.478986,0.388081,2,0.693147,15.451548,4,1.386294,0.463693,-6.982344e+10,3
3430610,151424,3,14424,122398719,CA,20051122,168,552.29,1514243,6.314073,-0.139448,2,0.693147,15.451548,2,0.693147,-0.308816,-6.982344e+10,4
3473022,151424,3,14424,122398719,CA,20060711,168,552.29,1514243,6.314073,-0.139448,2,0.693147,15.451548,2,0.693147,-0.308816,-6.982344e+10,5


In [24]:
ranked = ranked.sort_values(by=['rank'])
ranked.head()

,Family ID,Member ID,Provider ID,Provider Type,State,Date,Procedure,Amount,Patient ID,amountLog,Amount Normed,patientCount,patientCountLog,Patient Count Normed,providerCount,providerCountLog,Provider Count Normed,Score,rank
1173274,265506,1,18293,122398719,NY,20060605,153,976.19,2655061,6.883657,0.188191,2,0.693147,10.062427,90,4.499810,1.526027,-8.637489e+10,1
1369479,145964,0,12387,122398757,NY,20060713,96,1145.87,1459640,7.043919,0.525008,2,0.693147,0.979908,58,4.060443,0.912202,-4.578670e+09,1
3144909,445989,2,18282,122398769,FL,20060712,118,1123.62,4459892,7.024311,0.545113,2,0.693147,17.428449,116,4.753590,1.131069,-1.486294e+11,1
768043,201129,1,5395,122398782,FL,20051112,169,921.09,2011291,6.825558,0.911336,2,0.693147,12.020958,165,5.105945,0.680856,-1.149867e+11,1
1378128,497848,3,9709,122398794,TX,20051113,90,438.06,4978483,6.082356,-2.253155,1,0.000000,-0.612213,65,4.174387,0.753640,-1.294986e+10,1


In [25]:
ranked = ranked.drop(ranked[ranked['rank'] > 100].index)

In [31]:
ranked = ranked[['Family ID', 'Member ID', 'Provider ID', 'Date', 'Provider Type', 'rank']]
ranked.head()

,Family ID,Member ID,Provider ID,Date,Provider Type,rank
1173274,265506,1,18293,20060605,122398719,1
1369479,145964,0,12387,20060713,122398757,1
3144909,445989,2,18282,20060712,122398769,1
768043,201129,1,5395,20051112,122398782,1
1378128,497848,3,9709,20051113,122398794,1


In [32]:
ranked.to_csv('file2.csv')